In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
original_df = pd.read_csv("heart.csv")

In [3]:
print("The shape of the dataset is : ", original_df.shape)

The shape of the dataset is :  (303, 14)


In [4]:
original_df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
generated_df = pd.read_csv("generated.csv", header=None)

In [6]:
print("The shape of the dataset is : ", generated_df.shape)

The shape of the dataset is :  (303, 14)


In [7]:
generated_df.columns = original_df.columns

In [8]:
generated_df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,58.587723,0.0,2.0,126.208786,306.10492,0.0,1.0,165.07751,0.0,0.059414,2.0,1.0,2.0,1.0
1,59.176660,1.0,2.0,134.346220,245.68758,0.0,0.0,155.57610,0.0,0.824780,1.0,0.0,3.0,1.0
2,52.082695,0.0,2.0,127.719210,277.77197,0.0,0.0,155.61465,0.0,0.160931,2.0,1.0,2.0,1.0
3,49.808850,1.0,0.0,125.714230,218.88461,0.0,1.0,147.57452,1.0,0.223681,2.0,0.0,3.0,1.0
4,47.450340,0.0,2.0,121.424090,217.67462,0.0,1.0,159.43839,0.0,0.773253,1.0,0.0,2.0,1.0


##### Количество уникальных значений

In [9]:
dict = {}
for i in list(original_df.columns):
    dict[i] = original_df[i].value_counts().shape[0]

pd.DataFrame(dict, index=["unique count"]).transpose()

,unique count
age,41
sex,2
cp,4
trtbps,49
chol,152
fbs,2
restecg,3
thalachh,91
exng,2
oldpeak,40


In [10]:
dict = {}
for i in list(generated_df.columns):
    dict[i] = generated_df[i].value_counts().shape[0]

pd.DataFrame(dict, index=["unique count"]).transpose()

,unique count
age,303
sex,2
cp,4
trtbps,303
chol,303
fbs,2
restecg,2
thalachh,303
exng,2
oldpeak,303


##### Разделение на непрерывные и дискретные столбцы

In [11]:
cat_cols = ['sex','exng','caa','cp','fbs','restecg','slp','thall']
con_cols = ["age","trtbps","chol","thalachh","oldpeak"]
target_col = ["output"]
print("The categorial cols are : ", cat_cols)
print("The continuous cols are : ", con_cols)
print("The target variable is :  ", target_col)

The categorial cols are :  ['sex', 'exng', 'caa', 'cp', 'fbs', 'restecg', 'slp', 'thall']
The continuous cols are :  ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']
The target variable is :   ['output']


##### Общая статистика

In [12]:
original_df[con_cols].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,303.0,54.366337,9.082101,29.0,47.5,55.0,61.0,77.0
trtbps,303.0,131.623762,17.538143,94.0,120.0,130.0,140.0,200.0
chol,303.0,246.264026,51.830751,126.0,211.0,240.0,274.5,564.0
thalachh,303.0,149.646865,22.905161,71.0,133.5,153.0,166.0,202.0
oldpeak,303.0,1.039604,1.161075,0.0,0.0,0.8,1.6,6.2


In [13]:
generated_df[con_cols].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,303.0,54.899178,7.513066,35.559757,49.872359,55.539700,60.458090,72.521910
trtbps,303.0,131.910740,7.803876,109.531720,126.461265,132.444400,137.080705,163.263400
chol,303.0,249.983379,30.928561,188.491000,228.209350,244.448650,265.435835,366.394130
thalachh,303.0,155.102857,12.816038,118.177315,147.508925,156.284820,163.658235,187.336460
oldpeak,303.0,0.747483,0.630911,0.005131,0.247585,0.584833,1.127620,4.186105


##### Импорт библиотек

In [14]:
# Scaling
from sklearn.preprocessing import RobustScaler

# Train Test Split
from sklearn.model_selection import train_test_split

# Models
import torch
import torch.nn as nn
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.dummy import DummyClassifier

# Metrics
from sklearn.metrics import f1_score

##### Предобработка

In [15]:
df1 = original_df

# define the columns to be encoded and scaled
cat_cols = ['sex','exng','caa','cp','fbs','restecg','slp','thall']
con_cols = ["age","trtbps","chol","thalachh","oldpeak"]

# encoding the categorical columns
df1 = pd.get_dummies(df1, columns = cat_cols, drop_first = True)

# defining the features and target
original_X = df1.drop(['output'],axis=1)
original_y = df1[['output']]

# instantiating the scaler
scaler = RobustScaler()

# scaling the continuous featuree
original_X[con_cols] = scaler.fit_transform(original_X[con_cols])
print("The first 5 rows of original_X are")
original_X.head()

The first 5 rows of original_X are


,age,trtbps,chol,thalachh,oldpeak,sex_1,exng_1,caa_1,caa_2,caa_3,...,cp_2,cp_3,fbs_1,restecg_1,restecg_2,slp_1,slp_2,thall_1,thall_2,thall_3
0,0.592593,0.75,-0.110236,-0.092308,0.9375,1,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
1,-1.333333,0.00,0.157480,1.046154,1.6875,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
2,-1.037037,0.00,-0.566929,0.584615,0.3750,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,0.074074,-0.50,-0.062992,0.769231,0.0000,1,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
4,0.148148,-0.50,1.795276,0.307692,-0.1250,0,1,0,0,0,...,0,0,0,1,0,0,1,0,1,0


In [16]:
df2 = generated_df
df2[cat_cols] = df2[cat_cols].astype(int)

# encoding the categorical columns
df2 = pd.get_dummies(df2, columns = cat_cols, drop_first = True)
df1_not_df2 = df1.columns.difference(df2.columns)
df2[df1_not_df2] = 0

# defining the features and target
generated_X = df2.drop(['output'],axis=1)
generated_y = df2[['output']]

# instantiating the scaler
scaler = RobustScaler()

# scaling the continuous featuree
generated_X[con_cols] = scaler.fit_transform(generated_X[con_cols])
print("The first 5 rows of generated_X are")
generated_X.head()

The first 5 rows of generated_X are


,age,trtbps,chol,thalachh,oldpeak,sex_1,exng_1,caa_1,caa_2,caa_3,...,cp_3,fbs_1,restecg_1,slp_1,slp_2,thall_2,thall_3,caa_4,restecg_2,thall_1
0,0.287937,-0.587189,1.656247,0.544462,-0.597043,0,0,1,0,0,...,0,0,1,0,1,1,0,0,0,0
1,0.343572,0.179089,0.033281,-0.043885,0.272656,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,-0.326572,-0.444957,0.895151,-0.041498,-0.481687,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,0
3,-0.541375,-0.633759,-0.686716,-0.539361,-0.410384,1,1,0,0,0,...,0,0,1,0,1,0,1,0,0,0
4,-0.764176,-1.037749,-0.719220,0.195276,0.214105,0,0,0,0,0,...,0,0,1,1,0,1,0,0,0,0


##### Разделение на трейн и тест

In [17]:
original_X_train, original_X_test, original_y_train, original_y_test = \
train_test_split(original_X, original_y, test_size = 0.2, random_state = 42)
print("The shape of original_X_train is ", original_X_train.shape)
print("The shape of original_X_test is ", original_X_test.shape)
print("The shape of original_y_train is ", original_y_train.shape)
print("The shape of original_y_test is ", original_y_test.shape)

The shape of original_X_train is  (242, 22)
The shape of original_X_test is  (61, 22)
The shape of original_y_train is  (242, 1)
The shape of original_y_test is  (61, 1)


In [18]:
generated_X_train, generated_X_test, generated_y_train, generated_y_test = \
train_test_split(generated_X, generated_y, test_size = 0.2, random_state = 42)
print("The shape of generated_X_train is ", generated_X_train.shape)
print("The shape of generated_X_test is ", generated_X_test.shape)
print("The shape of generated_y_train is ", generated_y_train.shape)
print("The shape of generated_y_test is ", generated_y_test.shape)

The shape of generated_X_train is  (242, 22)
The shape of generated_X_test is  (61, 22)
The shape of generated_y_train is  (242, 1)
The shape of generated_y_test is  (61, 1)


##### Запись результатов

In [19]:
results = []
columns=['Модель', 'Качество на исходном датасете', 'Качество на сгенерированном датасете']

##### SVM

In [20]:
# instantiating the object and fitting
original_clf = SVC(kernel='linear', C=1, random_state=42).fit(original_X_train, original_y_train)

# predicting the values
y_pred = original_clf.predict(original_X_test)

original_accuracy = f1_score(original_y_test, y_pred)

# printing the test accuracy
print("The test original accuracy score of SVM is ", original_accuracy)

The test original accuracy score of SVM is  0.870967741935484


In [21]:
# instantiating the object and fitting
generated_clf = SVC(kernel='linear', C=1, random_state=42).fit(generated_X_train, generated_y_train)

# predicting the values
y_pred = generated_clf.predict(original_X_test)

generated_accuracy = f1_score(original_y_test, y_pred)

# printing the test accuracy
print("The test generated accuracy score of SVM is ", generated_accuracy)

The test generated accuracy score of SVM is  0.7692307692307693


In [22]:
results.append(['SVM', original_accuracy, generated_accuracy])

##### Логистическая регрессия

In [23]:
# instantiating the object
original_logreg = LogisticRegression()

# fitting the object
original_logreg.fit(original_X_train, original_y_train)

# calculating the probabilities
y_pred_proba = original_logreg.predict_proba(original_X_test)

# finding the predicted valued
y_pred = np.argmax(y_pred_proba,axis=1)

original_accuracy = f1_score(original_y_test, y_pred)

# printing the test accuracy
print("The test original accuracy score of Logistric Regression is ", original_accuracy)

The test original accuracy score of Logistric Regression is  0.9032258064516129


In [24]:
# instantiating the object
generated_logreg = LogisticRegression()

# fitting the object
generated_logreg.fit(generated_X_train, generated_y_train)

# calculating the probabilities
y_pred_proba = generated_logreg.predict_proba(original_X_test)

# finding the predicted valued
y_pred = np.argmax(y_pred_proba,axis=1)

generated_accuracy = f1_score(original_y_test, y_pred)

# printing the test accuracy
print("The test generated accuracy score of Logistric Regression is ", generated_accuracy)

The test generated accuracy score of Logistric Regression is  0.7317073170731707


In [25]:
results.append(['Логистическая регрессия', original_accuracy, generated_accuracy])

##### Decision Tree

In [26]:
# instantiating the object
original_dt = DecisionTreeClassifier(random_state = 42)

# fitting the model
original_dt.fit(original_X_train, original_y_train)

# calculating the predictions
y_pred = original_dt.predict(original_X_test)

original_accuracy = f1_score(original_y_test, y_pred)

# printing the test accuracy
print("The test original accuracy score of Decision Tree is ", original_accuracy)

The test original accuracy score of Decision Tree is  0.7796610169491525


In [27]:
# instantiating the object
generated_dt = DecisionTreeClassifier(random_state = 42)

# fitting the model
generated_dt.fit(generated_X_train, generated_y_train)

# calculating the predictions
y_pred = generated_dt.predict(original_X_test)

generated_accuracy = f1_score(original_y_test, y_pred)

# printing the test accuracy
print("The test generated accuracy score of Decision Tree is ", generated_accuracy)

The test generated accuracy score of Decision Tree is  0.6329113924050632


In [28]:
results.append(['Decision Tree', original_accuracy, generated_accuracy])

##### Random Forest

In [29]:
# instantiating the object
original_rf = RandomForestClassifier()

# fitting the model
original_rf.fit(original_X_train, original_y_train)

# calculating the predictions
y_pred = original_rf.predict(original_X_test)

original_accuracy = f1_score(original_y_test, y_pred)

# printing the test accuracy
print("The test original accuracy score of Random Forest is ", original_accuracy)

The test original accuracy score of Random Forest is  0.8333333333333334


In [30]:
# instantiating the object
generated_rf = RandomForestClassifier()

# fitting the model
generated_rf.fit(generated_X_train, generated_y_train)

# calculating the predictions
y_pred = generated_rf.predict(original_X_test)

generated_accuracy = f1_score(original_y_test, y_pred)

# printing the test accuracy
print("The test generated accuracy score of Random Forest is ", generated_accuracy)

The test generated accuracy score of Random Forest is  0.6813186813186813


In [31]:
results.append(['Random Forest', original_accuracy, generated_accuracy])

##### Gradient Boosting Classifier 

In [32]:
# instantiate the classifier
original_gbt = GradientBoostingClassifier(n_estimators = 300,
                                          max_depth=1,
                                          subsample=0.8,
                                          max_features=0.2,
                                          random_state=42)

# fitting the model
original_gbt.fit(original_X_train, original_y_train)

# predicting values
y_pred = original_gbt.predict(original_X_test)

original_accuracy = f1_score(original_y_test, y_pred)

print("The test original accuracy score of Gradient Boosting Classifier is ", original_accuracy)

The test original accuracy score of Gradient Boosting Classifier is  0.870967741935484


In [33]:
# instantiate the classifier
generated_gbt = GradientBoostingClassifier(n_estimators = 300,
                                           max_depth=1,
                                           subsample=0.8,
                                           max_features=0.2,
                                           random_state=42)

# fitting the model
generated_gbt.fit(generated_X_train, generated_y_train)

# predicting values
y_pred = generated_gbt.predict(original_X_test)

generated_accuracy = f1_score(original_y_test, y_pred)

print("The test generated accuracy score of Gradient Boosting Classifier is ", generated_accuracy)

The test generated accuracy score of Gradient Boosting Classifier is  0.7532467532467533


In [34]:
results.append(['Gradient Boosting Classifier', original_accuracy, generated_accuracy])

#### Baseline models

In [35]:
# instantiate the classifier
original_dummy_clf = DummyClassifier(strategy="uniform", random_state=42)

# fitting the model
original_dummy_clf.fit(original_X_train, original_y_train)

# predicting values
y_pred = original_dummy_clf.predict(original_X_test)

original_accuracy = f1_score(original_y_test, y_pred)

print("The test original accuracy score of Uniform Dummy Classifier is ", original_accuracy)

The test original accuracy score of Uniform Dummy Classifier is  0.5079365079365079


In [36]:
# instantiate the classifier
generated_dummy_clf = DummyClassifier(strategy="uniform", random_state=42)

# fitting the model
generated_dummy_clf.fit(generated_X_train, generated_y_train)

# predicting values
y_pred = generated_dummy_clf.predict(original_X_test)

generated_accuracy = f1_score(original_y_test, y_pred)

print("The test generated accuracy score of Uniform Dummy Classifier is ", generated_accuracy)

The test generated accuracy score of Uniform Dummy Classifier is  0.5079365079365079


In [37]:
results.append(['Uniform Dummy Classifier', original_accuracy, generated_accuracy])

In [50]:
# instantiate the classifier
original_dummy_clf = DummyClassifier(strategy="most_frequent")

# fitting the model
original_dummy_clf.fit(original_X_train, original_y_train)

# predicting values
y_pred = original_dummy_clf.predict(original_X_test)

original_accuracy = f1_score(original_y_test, y_pred)

print("The test original accuracy score of Most Frequent Dummy Classifier is ", original_accuracy)

The test original accuracy score of Most Frequent Dummy Classifier is  0.6881720430107526


In [51]:
# instantiate the classifier
generated_dummy_clf = DummyClassifier(strategy="most_frequent")

# fitting the model
generated_dummy_clf.fit(generated_X_train, generated_y_train)

# predicting values
y_pred = generated_dummy_clf.predict(original_X_test)

generated_accuracy = f1_score(original_y_test, y_pred)

print("The test generated accuracy score of Most Frequent Dummy Classifier is ", generated_accuracy)

The test generated accuracy score of Most Frequent Dummy Classifier is  0.6881720430107526


In [52]:
results.append(['Most Frequent Dummy Classifier', original_accuracy, generated_accuracy])

##### Результаты

In [41]:
table = pd.DataFrame(data=results, columns=columns)

In [42]:
table

,Модель,Качество на исходном датасете,Качество на сгенерированном датасете
0,SVM,0.870968,0.769231
1,Логистическая регрессия,0.903226,0.731707
2,Decision Tree,0.779661,0.632911
3,Random Forest,0.833333,0.681319
4,Gradient Boosting Classifier,0.870968,0.753247
5,Uniform Dummy Classifier,0.507937,0.507937
6,Most Frequent Dummy Classifier,0.688172,0.688172
